In [1]:
from datetime import datetime
from dataclasses import dataclass
import mariadb

#fake_date = datetime(1,1,1)
@dataclass
class GameRecord:
    event: str = ""
    site: str  = ""
    game_date: datetime.date = None
    round: int = None
    result: int = None
    event_date: datetime.date = None
    event_sponsor: str = ""
    section: str = ""
    stage: str = ""
    board: int = None
    opening: str = None
    variation: str = None
    subvariation: str = None
    eco: str = ""
    nic: str = ""
    game_time: datetime.time = None
    game_utc_date: datetime.date = None
    game_utc_time: datetime.time = None
    time_control: str = ""
    setup: int = None
    fen: str = ""
    termination: str = ""
    annotator: str = ""
    mode: str = ""
    plycount: int = None
    white_player_id: int = None
    black_player_id: int = None
    
def connect():
    global conn
    global cursor

    if conn.open:
        return
        
    # Database connection details
    db_config = {
        'user': 'chess_user',
        'password': 'billich',
        'host': 'localhost',
        'database': 'chess',
        'port': 3306  # Standard port for MariaDB
    }
    # Establishing the connection
    conn = mariadb.connect(**db_config)
    # Create a cursor to execute queries
    cursor = conn.cursor()

def disconnect():
    cursor.close()
    conn.close()

def commit():
    conn.commit()
    
def rollback():
    conn.rollback()
    
def lookup_player_by_name(p_name):
    if p_name == '':
        return None
    
    cursor.execute("select id from player where name=?", (p_name,))
    result = cursor.fetchall()

    if not result:
        return None

    return result[0][0]

def insert_player(p_name, p_fide_id = 0):
    if p_name == '':
        return None    

    if (p_fide_id == 0):
        cursor.execute("insert into player (name) values (?) on duplicate key update id=id returning id", (p_name,))
    else:
        cursor.execute("insert into player (name, fide_id) values (?, ?) on duplicate key update fide_id=? returning id", (p_name, p_fide_id, p_fide_id,))

    result = cursor.fetchall()
    #conn.commit()
    
    return result[0][0]   
                       
def insert_elo (p_player_id = 0, p_elo_num = 0, p_elo_date = None):
    if (p_player_id == 0 or p_elo_num == 0 or p_elo_date == None):
        return None

    cursor.execute("insert into elo (player_id, elo_num, elo_date) values (?, ?, ?) on duplicate key update elo_num=? returning player_id", (p_player_id, p_elo_num, p_elo_date.strftime('%Y-%m-%d'), p_elo_num,))
    
    result = cursor.fetchall()
    #conn.commit()
    
    return result[0][0] 

def insert_title (p_player_id = 0, p_title = "", p_title_date = None):
    if (p_player_id == 0 or p_title == "" or p_title_date == None):
        return None

    cursor.execute("insert into title (player_id, title, title_date) values (?, ?, ?) on duplicate key update title=? returning player_id", (p_player_id, p_title, p_title_date.strftime('%Y-%m-%d'), p_title,))
    
    result = cursor.fetchall()
    #conn.commit()
    
    return result[0][0]     

def insert_game (p_game_record):
    # one and only one player has to be set
    if ((p_game_record.white_player_id == None and p_game_record.black_player_id == None)
        or 
       (p_game_record.white_player_id != None and p_game_record.black_player_id != None)):
        return None

    cursor.execute("insert into game (event, site, game_date, round, result, event_date, event_sponsor, section, stage, board, opening, variation, "
                   "subvariation, eco, nic, game_time, game_utc_date, game_utc_time, time_control, setup, fen, termination, annotator, mode, "
                   "plycount, white_player_id, black_player_id) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) returning id",
                   (p_game_record.event, p_game_record.site, p_game_record.game_date, p_game_record.round, p_game_record.result, p_game_record.event_date, 
                    p_game_record.event_sponsor, p_game_record.section, p_game_record.stage, p_game_record.board, p_game_record.opening, p_game_record.variation, 
                   p_game_record.subvariation, p_game_record.eco, p_game_record.nic, p_game_record.game_time, p_game_record.game_utc_date, 
                   p_game_record.game_utc_time, p_game_record.time_control, p_game_record.setup, p_game_record.fen, p_game_record.termination, 
                    p_game_record.annotator, p_game_record.mode, p_game_record.plycount, p_game_record.white_player_id, p_game_record.black_player_id,))

    result = cursor.fetchall()
    #conn.commit()
    
    return result[0][0]     
    

In [79]:
connect()
#id=lookup_player_by_name("Ado lf")
id=insert_player("Adolf", 123)
print (id)

1


In [45]:
connect()
game = GameRecord(event="Hastings2", white_player_id=None, black_player_id=1)
id=insert_game(game)
print(id)

2


In [14]:
conn.rollback()